In [ ]:
# Step 1: Data Acquisition
# Download the Plant disease dataset from Kaggle

# Install kaggle
!pip install kaggle

# Mount the Google drive so that you can store your Kaggle API credentials
from google.colab import drive
drive.mount('/content/drive')

# Create a Kaggle directory
!mkdir -p ~/.kaggle

# Copy the kaggle.json file to the Kaggle directory
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/

# Set permissions for the Kaggle API key
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset using the Kaggle API
!kaggle datasets download -d emmarex/plantdisease

# Unzip the dataset (if needed)
!unzip plantdisease.zip

In [ ]:
# Step 2: Loading and augment the dataset(Augmentation: Techniques applied to increase dataset diversity, helping the model generalize better)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load dataset
data_dir = 'PlantVillage/'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Data preprocessing for validation set (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Create validation data generator
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# Step 3: Use Transfer learning with a pre-trained model
# Model Selection
# Utilize ResNet50, a well-established convolutional neural network known for its effectiveness in image classification tasks, particularly in plant disease detection

from tensorflow.keras.applications import ResNet50  # ResNet50 is a popular deep learning model that is 50 layers deep
from tensorflow.keras.models import Model # Class used to define a 'Keras' model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Define a base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding a custom layer
x = base_model.output # Take the output of the base_model ResNet50
x = GlobalAveragePooling2D()(x) # This layer applies global average pooling to the feature map

# Adding fully connected layers
x = Dense(256, activation='relu')(x)  # Dense layer for helping the model learn complex patterns
predictions = Dense(15, activation='softmax')(x)  # Another fully connected layer with 38 neurons(classes). The softmax activation function converts the output into praobabilities for each class so that they sum to 1, suitable for multi classification problems

# Creating the model
model = Model(inputs=base_model.input, outputs=predictions) # inputs: base_model input layer and output:'predictions', which contain the probabilities of each class

In [4]:
# Step 4: Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 5: Training the model using augmented data while validating it against a seperate validation test
model.fit(train_generator, epochs=10, validation_data=validation_generator)

In [ ]:
# Step 6: Fine-tuning the model, by unfreezing some of the deeper layers of ResNet50
for layer in base_model.layers[-10:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=validation_generator)


In [ ]:
# Step 7: Upload the image and predict the class of the image with the model
from google.colab import files
# Import the Image module instead of the Image class
from PIL import Image as PILImage
import numpy as np
from keras.preprocessing import image
from IPython.display import display, Image

# Upload image
uploaded = files.upload()

# Get the filename of the uploaded image
img_name = list(uploaded.keys())[0]

# Open and preprocess the image
# Use PILImage.open instead of Image.open
img = PILImage.open(img_name)
img = img.resize((224, 224))  # Resize to match model input size
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Display the image
display(Image(filename=img_name))

# Perform prediction
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction)

# Map predicted class index to class name
class_names = {v: k for k, v in train_generator.class_indices.items()}
predicted_class_name = class_names[predicted_class]
print(f"Predicted class: {predicted_class_name}")